In [1]:
from nltk.tokenize.casual import casual_tokenize
import gensim
from gensim import corpora
import re
import pandas as pd
import numpy as np
from string import punctuation
from gensim.models import CoherenceModel
import os

input_filepath="../../data/processed/"
filename="trisma2016_cleaned_text_by_lga_and_time_of_day.csv"

In [2]:
target = pd.read_csv(input_filepath+"cleaned_target.csv")
nswdf_target = target[(target["Type"]=='Rate per 100,000 population') & (target['Year'] == 2016)].dropna()
nswdf_target = nswdf_target[['LGA', 'Assault - domestic violence', 'Assault - non-domestic violence', 'Fraud', 'Sexual Offences', 'Malicious damage to property', 'Prohibited weapons offences',]]

In [3]:
target = pd.read_csv(input_filepath+"cleaned_target.csv")
nswdf_target = target[(target["Type"]=='Rate per 100,000 population') & (target['Year'] == 2016)].dropna()
nswdf_target = nswdf_target[['LGA', 'Assault - domestic violence', 'Assault - non-domestic violence', 'Fraud', 'Sexual Offences', 'Malicious damage to property', 'Prohibited weapons offences',]]
nswdf_target = nswdf_target.set_index("LGA")
nswdf_target = np.log(nswdf_target)
nswdf_target

,Assault - domestic violence,Assault - non-domestic violence,Fraud,Sexual Offences,Malicious damage to property,Prohibited weapons offences
LGA,,,,,,
Albury,6.299733,6.334338,6.263887,5.378146,7.181636,5.756704
Armidale Regional,6.217540,6.481927,6.734923,5.398352,7.703918,5.659568
Ballina,5.544054,5.772687,6.223344,5.042361,6.678706,5.027093
Bathurst Regional,6.084028,6.136396,6.531605,5.300932,7.076881,5.422629
Bayside,5.755702,5.613635,6.590862,4.448712,6.371043,4.687338
Bega Valley,5.743838,5.833989,5.762531,5.031643,6.697170,5.031643
Bellingen,5.632005,5.574846,5.514222,4.756536,6.405194,4.533393
Berrigan,5.164719,4.941576,5.095726,4.402579,6.551014,5.634723
Blacktown,6.387513,6.116154,6.710620,5.099780,6.888318,5.002186


In [4]:


tweetdf=pd.read_csv(os.path.join(input_filepath, filename),header=0)
tweetdf

,Unnamed: 0,lga,time_of_day,text,cleaned
0,0,Albury,afternoon,"I can't say if its ""personally directed"" @ me ...",can't say if its personally directed me but wh...
1,1,Albury,evening,New year resolution \n\nFind another job so yo...,new year resolution find another job so you do...
2,2,Albury,morning,@BluesTumut @Group9RL going to be a smashing s...,going to be smashing season this year good luc...
3,3,Albury,night,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
4,4,Armidale Regional,afternoon,#تنفيذ_القصاص_على_47_ارهابي\n\nليسوا على مذهب ...,#tnfydh_lqss_ _47_rhby lysw mdhhb whd wl jnsy ...
5,5,Armidale Regional,evening,@Photos_KP bit of a rare site over there.. @Ph...,bit of rare site over there can't it's overcas...
6,6,Armidale Regional,morning,@RebeccaRacer woohoo! I think there's also a n...,woohoo think there's also number of people exc...
7,7,Armidale Regional,night,Tranquility in the Park. @ Moore Park Inn Bout...,tranquility in the park moore park inn boutiqu...
8,8,Ballina,afternoon,When my lil sister calls her close friends cou...,when my lil sister calls her close friends cou...
9,9,Ballina,evening,If you are feeling too positive about yourself...,if you are feeling too positive about yourself...


In [6]:
#now split by various doc types and run LGA. DO i need to write out interim files? Not really, not yet

by_lga = tweetdf
by_lga = by_lga.set_index("lga")
by_lga

,Unnamed: 0,time_of_day,text,cleaned
lga,,,,
Albury,0,afternoon,"I can't say if its ""personally directed"" @ me ...",can't say if its personally directed me but wh...
Albury,1,evening,New year resolution \n\nFind another job so yo...,new year resolution find another job so you do...
Albury,2,morning,@BluesTumut @Group9RL going to be a smashing s...,going to be smashing season this year good luc...
Albury,3,night,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
Armidale Regional,4,afternoon,#تنفيذ_القصاص_على_47_ارهابي\n\nليسوا على مذهب ...,#tnfydh_lqss_ _47_rhby lysw mdhhb whd wl jnsy ...
Armidale Regional,5,evening,@Photos_KP bit of a rare site over there.. @Ph...,bit of rare site over there can't it's overcas...
Armidale Regional,6,morning,@RebeccaRacer woohoo! I think there's also a n...,woohoo think there's also number of people exc...
Armidale Regional,7,night,Tranquility in the Park. @ Moore Park Inn Bout...,tranquility in the park moore park inn boutiqu...
Ballina,8,afternoon,When my lil sister calls her close friends cou...,when my lil sister calls her close friends cou...


In [6]:
#by_lga.apply(lambda x: len(x['text']), axis=1)

In [11]:
punc_regex = re.compile('[%s]' % re.escape(punctuation))

#finds best ntopics
def run_lda(text_series):
    toks = [x.split(" ") for x in text_series] # join into a 2d list
    
    dictionary = corpora.Dictionary(toks)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in toks]

    Lda = gensim.models.ldamodel.LdaModel

    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    return ldamodel
    #model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=toks, start=start, limit=stop, step=step)
    #return model_list, coherence_values



In [16]:
## reference: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#17howtofindtheoptimalnumberoftopicsforlda

def compute_coherence_values(dictionary, corpus, texts, start, step, limit):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=50)
        print(".")
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [10]:
ldams = run_lda(by_lga['text'])



TypeError: run_lda() missing 3 required positional arguments: 'start', 'step', and 'stop'

In [ ]:
toks = [x.split(" ") if (not pd.isnull(x)) else [''] for x in by_lga["cleaned"] ]
dictionary = corpora.Dictionary(toks)
corpus = [dictionary.doc2bow(doc) for doc in toks]

model = gensim.models.ldamodel.LdaModel(corpus, num_topics=300, id2word = dictionary, passes=50)

In [ ]:
import pickle
with open("todldamodel.pk", 'wb') as f:
    pickle.dump(model, f)

In [52]:
model.show_topics(100)

[(246,
  'nan*"zhl" + nan*"ybhth" + nan*"abbas" + nan*"#singwhenyouredrawing" + nan*"#ohfcb" + nan*"#neweyisskyblue" + nan*"#cleansheets" + nan*"disgracing" + nan*"yjdty" + nan*"wml"'),
 (155,
  'nan*"zhl" + nan*"ybhth" + nan*"abbas" + nan*"#singwhenyouredrawing" + nan*"#ohfcb" + nan*"#neweyisskyblue" + nan*"#cleansheets" + nan*"disgracing" + nan*"yjdty" + nan*"wml"'),
 (185,
  'nan*"zhl" + nan*"ybhth" + nan*"abbas" + nan*"#singwhenyouredrawing" + nan*"#ohfcb" + nan*"#neweyisskyblue" + nan*"#cleansheets" + nan*"disgracing" + nan*"yjdty" + nan*"wml"'),
 (277,
  'nan*"zhl" + nan*"ybhth" + nan*"abbas" + nan*"#singwhenyouredrawing" + nan*"#ohfcb" + nan*"#neweyisskyblue" + nan*"#cleansheets" + nan*"disgracing" + nan*"yjdty" + nan*"wml"'),
 (113,
  'nan*"zhl" + nan*"ybhth" + nan*"abbas" + nan*"#singwhenyouredrawing" + nan*"#ohfcb" + nan*"#neweyisskyblue" + nan*"#cleansheets" + nan*"disgracing" + nan*"yjdty" + nan*"wml"'),
 (208,
  'nan*"zhl" + nan*"ybhth" + nan*"abbas" + nan*"#singwhenyoured

In [22]:
model.save("todldamodel.pk")

In [97]:
model = gensim.models.ldamodel.LdaModel.load("lgaldamodel.pk")

In [43]:
model.top_topics(corpus)

[([(0.00029454436, '#S2S'),
   (0.00029454436, 'https://t.co/Bc9KeakZCp'),
   (0.00029454436, 'sort,'),
   (0.00029454436, 'https://t.co/GvixdgCEQF'),
   (0.00029454436, 'https://t.co/i01AKjf6sI'),
   (0.00029454436, '@Woodleafarms…'),
   (0.00029454436, '@Mary_Raynes'),
   (0.00029454433, 'https://t.co/bSKJqnThk0'),
   (0.00029454433, 'https://t.co/9gHqcU8vDq'),
   (0.0002945443, '@Anonylox'),
   (0.0002945443, 'Office:'),
   (0.0002945443, 'Jindera).'),
   (0.00029454415, 'flowers...'),
   (0.00029454412, 'https://t.co/gYi4enPWDg'),
   (0.00029454412, 'amazed,'),
   (0.00029454412, 'https://t.co/na0oT33LZs'),
   (0.00029454406, 'https://t.co/927tcCaIOv'),
   (0.00029454406, '06:09'),
   (0.00029454372, 'Gerogery,'),
   (0.0002945437, 'https://t.co/PnJXS6M4Uv')],
  1.3100009964825002e-10),
 ([(0.00086490315, 'https://t.co/bOAsOpJkhi'),
   (0.0005769221, 'Pidgeot'),
   (0.00057692203, 'potatoes….'),
   (0.00057692203, '🐾☺️'),
   (0.00057692203, 'catalyst/quantum)?'),
   (0.00057692203,

In [33]:
#classify lgas
inferred = model.inference(corpus)
inferred = inferred[0]

In [35]:
import numpy as np
inferdf = pd.DataFrame(inferred)


In [36]:
inferdf.index = by_lga.index
inferdf

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
lga,,,,,,,,,,,,,,,,,,,,,
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albury,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import probplot
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt

def kfold_cv(k, crimenum, dataframe, target):
    lga_names = target.index
    start_col = target.columns.get_loc('Assault - domestic violence')
    kf = KFold(n_splits=5, shuffle=True)
    models = []
    rmse = []
    r2 = []
    allpreds = target.iloc[0:0]
    allpreds["preds"] = pd.Series()
    output = pd.DataFrame(index=range(len(dataframe)))
    for train_index, test_index in kf.split(lga_names):
        train_lgas = lga_names[train_index]
        traindf = dataframe.loc[train_lgas]
        train_target = target.loc[train_lgas]
        
        test_lgas = lga_names[test_index]
        testdf = dataframe.loc[test_lgas]
        test_target = target.loc[test_lgas].copy()
        #import pdb; pdb.set_trace()
        model = RandomForestRegressor()
        model.fit(traindf, train_target.iloc[:,start_col+crimenum])
        models.append(model)
        #print(lga_loc)
        preds = model.predict(testdf)
        test_target["preds"] = preds
        #import pdb; pdb.set_trace()
        allpreds = pd.concat([allpreds, test_target])
        #is this a fair measure
        rmse.append(np.sqrt(mean_squared_error(preds, test_target.iloc[:,start_col+crimenum])))
        r2.append(r2_score(test_target.iloc[:,start_col+crimenum], preds))

    probplot(allpreds.iloc[:,start_col+crimenum] - allpreds.iloc[:,-1], dist='norm', plot=plt)
    #import pdb; pdb.set_trace()
    plt.show()
   
    rmse.append(np.sqrt(mean_squared_error(allpreds["preds"], allpreds.iloc[:,start_col+crimenum])))
    r2.append(r2_score(allpreds.iloc[:,start_col+crimenum], allpreds["preds"]))
    return output, rmse, r2, models, allpreds

outputs = []
for i in range(6):
    outputs.append(kfold_cv(5, 0, inferdf, nswdf_target))
    outputs[-1][1:3]

In [87]:
for i in range(len(outputs)):
    print(nswdf_target.columns[i])
    print("r2: ",outputs[i][2])
    print("rmse: ",outputs[i][1])

Assault - domestic violence
r2:  [-1.2081656094449986, -0.37514405633963466, 0.2700442985414264, 0.20085745357213314, -0.09948601994703354, -0.07567759886782599]
rmse:  [0.7906155479699832, 0.5215604222646973, 0.6008491035289741, 0.6233570914707449, 0.6657999069701667, 0.6465263248445073]
Assault - non-domestic violence
r2:  [0.04418189450746457, -0.3415394317591838, 0.17012877863745346, -0.06560151541897508, 0.2607213009733236, 0.04362917510972952]
rmse:  [0.5641180548083128, 0.7178715792148508, 0.5438472821657954, 0.6818130173297049, 0.5137775882044995, 0.6096187416846253]
Fraud
r2:  [0.14029948594649877, -0.3177005914950992, 0.25266264163889185, -0.20793357067167362, -0.5342685144475823, 0.037401935426443655]
rmse:  [0.6350716644691716, 0.6546365313596401, 0.6866210770663596, 0.5201899223212482, 0.5443924721240962, 0.6116002337579612]
Sexual Offences
r2:  [0.19099107353295264, 0.1316980528653422, -0.17897850707743723, -0.01835215539073598, 0.08031108185194502, 0.0584711001611794]
rm

In [93]:
imp = []
for i, output in enumerate(outputs):
    imp.append(np.zeros(300))
    for mod in output[3]:
        imp[-1] = imp[-1] + mod.feature_importances_
   
imp

[array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.26803976e-02,
        7.58872626e-02, 0.00000000e+00, 0.00000000e+00, 1.37384568e-03,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.02116070e-07,
        0.00000000e+00, 0.00000000e+00, 4.84786265e-03, 7.63805062e-03,
        3.12034216e-01, 0.00000000e+00, 6.61735306e-02, 0.00000000e+00,
        0.00000000e+00, 4.48710501e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 4.71785848e-09, 4.52155581e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.788418

In [98]:
#for each target get most important topics
for i, importvec in enumerate(imp):
    print(nswdf_target.columns[i])
    top10 = np.argsort(importvec)[:10]
    for j in top10:
        print(model.show_topic(j))
        

Assault - domestic violence
[('the', 1.3401003e-06), ('a', 1.2417485e-06), ('to', 1.2091601e-06), ('I', 1.1696644e-06), ('is', 1.1662474e-06), ('in', 1.1575127e-06), ('and', 1.152133e-06), ('of', 1.119592e-06), ('for', 1.1105031e-06), ('on', 1.1066485e-06)]
[('the', 1.3567841e-06), ('a', 1.31081e-06), ('to', 1.2447258e-06), ('Wind', 1.2279082e-06), ('of', 1.2203769e-06), ('Rain', 1.2120687e-06), ('mm.', 1.2104393e-06), ('kts', 1.2093574e-06), ('Barometer', 1.1962999e-06), ('today', 1.1841928e-06)]
[('the', 1.3864573e-06), ('°C.', 1.3498177e-06), ('Barometer', 1.3416466e-06), ('a', 1.3226538e-06), ('I', 1.3075337e-06), ('mm.', 1.3007905e-06), ('today', 1.2852133e-06), ('Rain', 1.2600212e-06), ('to', 1.25661e-06), ('in', 1.2553263e-06)]
[('#beautifulhuman', 0.00058673415), ('@BJPAbhi1', 0.0005867341), ('@NickChavezBH', 0.0005867339), ('Kurri).', 0.0005867318), ('https://t.co/bOJW0W2a0t', 0.00029385524), ('https://t.co/S8kYQao4N3', 0.0002938552), ('@appkliyeash', 0.0002938552), ('https://

[('the', 1.3864573e-06), ('°C.', 1.3498177e-06), ('Barometer', 1.3416466e-06), ('a', 1.3226538e-06), ('I', 1.3075337e-06), ('mm.', 1.3007905e-06), ('today', 1.2852133e-06), ('Rain', 1.2600212e-06), ('to', 1.25661e-06), ('in', 1.2553263e-06)]
[('#beautifulhuman', 0.00058673415), ('@BJPAbhi1', 0.0005867341), ('@NickChavezBH', 0.0005867339), ('Kurri).', 0.0005867318), ('https://t.co/bOJW0W2a0t', 0.00029385524), ('https://t.co/S8kYQao4N3', 0.0002938552), ('@appkliyeash', 0.0002938552), ('https://t.co/9KRWKER70v', 0.0002938552), ('https://t.co/mtcQJdKSqJ', 0.0002938552), ('-absolutely', 0.00029385518)]
[('the', 1.2718504e-06), ('a', 1.244884e-06), ('to', 1.2165718e-06), ('in', 1.1540138e-06), ('of', 1.1424976e-06), ('I', 1.1144673e-06), ('is', 1.1084952e-06), ('on', 1.1073445e-06), ('Rain', 1.1061818e-06), ('0.0', 1.1021989e-06)]
[('the', 1.3020247e-06), ('to', 1.2149172e-06), ('a', 1.1820521e-06), ('of', 1.1341945e-06), ('I', 1.1338375e-06), ('is', 1.1085627e-06), ('at', 1.0952336e-06), ('

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
x = range(start, stop, step)
plt.plot(x, coh)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()